# Face and Mask

In [1]:
import cv2
import matplotlib.pyplot as plt
import helpers
import numpy as np
import tensorflow as tf
import glob
import random
from retinaface import RetinaFace
from PIL import Image
from od_metrics.BoundingBox import BoundingBox
from od_metrics.BoundingBoxes import BoundingBoxes
from od_metrics.utils import BBFormat, BBType
from od_metrics.Evaluator import Evaluator
use_pickle = False

## Load Kaggle Dataset
Kaggle dataset has 853 images

In [2]:
# load images and associated faces
# this takes a second since we load with cv2
imgs_with_labels = helpers.load_kaggle_863('../kaggle_dataset_863')
imgs_with_labels = helpers.convert_kaggle_863_for_metrics(imgs_with_labels)

## Facial Recognition
We use retinaface to detect facial features

In [3]:
# Initialize facial detection
detector = RetinaFace(quality = "normal")

model[normal quality] init ..
model success !


In [4]:
dl_detection_results = {}
dl_detection_results['retinaface_mobilenet_alt'] = imgs_with_labels['bboxes'].clone()
detections = dl_detection_results['retinaface_mobilenet_alt']

In [5]:
#Detect faces and crop them, add the cropped Faces into cropImages

cropImages = []
cnt = 0
for img_name, img_data in imgs_with_labels['raw_data'].items():
    img = img_data['img']
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # using default threshold
    faces = detector.predict(rgb_img.astype(np.float32), threshold=0.7)
    
    #Crop faces and add them to cropImages
    img = Image.fromarray(rgb_img)
    cropFaces = []
    for face in faces:
        cropFaces.append(img.crop((face['x1'], face['y1'], face['x2'], face['y2'])))
        
    cropImages.append(cropFaces)
    
    cnt += 1
    if cnt%25 == 0: print(cnt)

25
50
75
100
125
150
175
200
225
250
275
300
325
350
375
400
425
450
475
500
525
550
575
600
625
650
675
700
725
750
775
800
825
850


## Mask Detection
we will now pass cropImages through the mask classifier

In [6]:
dim = 64

# I am passing in PIL images, this converts it to PIL
def resize_img(pic):
    image_array = tf.keras.preprocessing.image.img_to_array(pic)
    img = tf.convert_to_tensor(image_array, dtype = 'uint8')
    img = tf.image.resize(img, [dim, dim])
    img = img/127.5-1
    return img

In [7]:
# The neural network

def conv_model(num_blocks, rate, dim):
    
    input = tf.keras.layers.Input(shape=(dim,dim,3))
    
    x = tf.keras.layers.Conv2D(16, (16,16), strides=1)(input)
    x = tf.keras.layers.Dropout(rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
        
    for i in range(num_blocks-1):
        x = tf.keras.layers.Conv2D(8, (8,8), strides=1)(x)
        x = tf.keras.layers.Dropout(rate)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)
        
    x = tf.keras.layers.Conv2D(filters=2, kernel_size=1, strides=1)(x)
    x = tf.keras.layers.Dropout(rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.GlobalMaxPooling2D()(x)
    predictions = tf.keras.layers.Activation('softmax')(x)
    
    model = tf.keras.Model(inputs=input, outputs=predictions)
    print(model.summary())
    
    return model

In [8]:
num_blocks = 3
dropout = .1
model = conv_model(num_blocks,.1, dim)
model.load_weights('mask_classification_model_3_50.h5')
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 49, 49, 16)        12304     
_________________________________________________________________
dropout (Dropout)            (None, 49, 49, 16)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 49, 49, 16)        64        
_________________________________________________________________
activation (Activation)      (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 42, 42, 8)         8200      
_________________________________________________________________
dropout_1 (Dropout)          (None, 42, 42, 8)         0     

In [9]:
# Prediction function, passes in PIL image
def pred(pic):
    im = resize_img(pic) # Convert and resize PIL image to 64x64 tensor
    vals = model.predict(np.expand_dims(im,axis=0))
    print(f"Mask Val: {vals[0][0]}, Non-mask Val: {vals[0][1]}")
    prediction = np.argmax(vals,axis=1)
    print(f"Image predicted as {prediction}")

In [11]:
for img in cropImages:
    for face in img:
        pred(face)




Mask Val: 0.11774208396673203, Non-mask Val: 0.8822579383850098
Image predicted as [1]
Mask Val: 0.16685953736305237, Non-mask Val: 0.8331404328346252
Image predicted as [1]
Mask Val: 0.9826881289482117, Non-mask Val: 0.017311837524175644
Image predicted as [0]
Mask Val: 0.9902515411376953, Non-mask Val: 0.00974851194769144
Image predicted as [0]
Mask Val: 0.9862380027770996, Non-mask Val: 0.013762053102254868
Image predicted as [0]
Mask Val: 0.02943778783082962, Non-mask Val: 0.970562219619751
Image predicted as [1]
Mask Val: 0.9884029030799866, Non-mask Val: 0.011597082950174809
Image predicted as [0]
Mask Val: 0.9484004974365234, Non-mask Val: 0.051599498838186264
Image predicted as [0]
Mask Val: 0.8974366784095764, Non-mask Val: 0.10256330668926239
Image predicted as [0]
Mask Val: 0.9989006519317627, Non-mask Val: 0.0010993293253704906
Image predicted as [0]
Mask Val: 0.9941182136535645, Non-mask Val: 0.005881754215806723
Image predicted as [0]
Mask Val: 0.9972623586654663, Non-mas

Mask Val: 0.9372426867485046, Non-mask Val: 0.06275730580091476
Image predicted as [0]
Mask Val: 0.9896236062049866, Non-mask Val: 0.010376319289207458
Image predicted as [0]
Mask Val: 0.9769513010978699, Non-mask Val: 0.023048697039484978
Image predicted as [0]
Mask Val: 0.9526138305664062, Non-mask Val: 0.04738617688417435
Image predicted as [0]
Mask Val: 0.9011161923408508, Non-mask Val: 0.09888381510972977
Image predicted as [0]
Mask Val: 0.9920111298561096, Non-mask Val: 0.0079888254404068
Image predicted as [0]
Mask Val: 0.963561475276947, Non-mask Val: 0.03643854707479477
Image predicted as [0]
Mask Val: 0.9910849928855896, Non-mask Val: 0.00891498476266861
Image predicted as [0]
Mask Val: 0.949701189994812, Non-mask Val: 0.0502987764775753
Image predicted as [0]
Mask Val: 0.763577401638031, Non-mask Val: 0.2364225685596466
Image predicted as [0]
Mask Val: 0.9863807559013367, Non-mask Val: 0.013619279488921165
Image predicted as [0]
Mask Val: 0.3180828094482422, Non-mask Val: 0.

Mask Val: 0.981145977973938, Non-mask Val: 0.01885402388870716
Image predicted as [0]
Mask Val: 0.3413151800632477, Non-mask Val: 0.6586847901344299
Image predicted as [1]
Mask Val: 0.9999974966049194, Non-mask Val: 2.5190136057062773e-06
Image predicted as [0]
Mask Val: 0.23446667194366455, Non-mask Val: 0.7655332684516907
Image predicted as [1]
Mask Val: 0.9440913796424866, Non-mask Val: 0.05590864270925522
Image predicted as [0]
Mask Val: 0.996094286441803, Non-mask Val: 0.003905719378963113
Image predicted as [0]
Mask Val: 0.9816474318504333, Non-mask Val: 0.0183525700122118
Image predicted as [0]
Mask Val: 0.9964409470558167, Non-mask Val: 0.003559047356247902
Image predicted as [0]
Mask Val: 0.9389278888702393, Non-mask Val: 0.06107209622859955
Image predicted as [0]
Mask Val: 0.4115809500217438, Non-mask Val: 0.5884190797805786
Image predicted as [1]
Mask Val: 0.9885317087173462, Non-mask Val: 0.011468261480331421
Image predicted as [0]
Mask Val: 0.998454213142395, Non-mask Val:

Mask Val: 0.9139502048492432, Non-mask Val: 0.08604975044727325
Image predicted as [0]
Mask Val: 0.9710205793380737, Non-mask Val: 0.028979381546378136
Image predicted as [0]
Mask Val: 0.9959959983825684, Non-mask Val: 0.004004011396318674
Image predicted as [0]
Mask Val: 0.004057972691953182, Non-mask Val: 0.9959420561790466
Image predicted as [1]
Mask Val: 0.9951469302177429, Non-mask Val: 0.0048530977219343185
Image predicted as [0]
Mask Val: 0.9942280054092407, Non-mask Val: 0.005771946161985397
Image predicted as [0]
Mask Val: 0.9879442453384399, Non-mask Val: 0.012055687606334686
Image predicted as [0]
Mask Val: 0.09158682078123093, Non-mask Val: 0.9084131717681885
Image predicted as [1]
Mask Val: 0.5249241590499878, Non-mask Val: 0.4750758707523346
Image predicted as [0]
Mask Val: 0.8495666980743408, Non-mask Val: 0.15043333172798157
Image predicted as [0]
Mask Val: 0.9954828023910522, Non-mask Val: 0.004517235793173313
Image predicted as [0]
Mask Val: 0.9944567680358887, Non-ma

Mask Val: 0.0028333391528576612, Non-mask Val: 0.9971666932106018
Image predicted as [1]
Mask Val: 0.9949918389320374, Non-mask Val: 0.005008156877011061
Image predicted as [0]
Mask Val: 0.9721726179122925, Non-mask Val: 0.0278274305164814
Image predicted as [0]
Mask Val: 0.9966089725494385, Non-mask Val: 0.003391087055206299
Image predicted as [0]
Mask Val: 0.9905288815498352, Non-mask Val: 0.009471104480326176
Image predicted as [0]
Mask Val: 0.9948244094848633, Non-mask Val: 0.005175655707716942
Image predicted as [0]
Mask Val: 0.005679346155375242, Non-mask Val: 0.9943206310272217
Image predicted as [1]
Mask Val: 0.13070857524871826, Non-mask Val: 0.869291365146637
Image predicted as [1]
Mask Val: 0.05134011432528496, Non-mask Val: 0.9486599564552307
Image predicted as [1]
Mask Val: 0.8955184817314148, Non-mask Val: 0.10448157042264938
Image predicted as [0]
Mask Val: 0.9238327145576477, Non-mask Val: 0.07616731524467468
Image predicted as [0]
Mask Val: 0.9036445021629333, Non-mask

...

Mask Val: 0.9952453970909119, Non-mask Val: 0.004754617810249329
Image predicted as [0]
Mask Val: 0.9936320185661316, Non-mask Val: 0.006367952562868595
Image predicted as [0]
Mask Val: 0.9815797209739685, Non-mask Val: 0.018420279026031494
Image predicted as [0]
Mask Val: 0.9853654503822327, Non-mask Val: 0.01463454682379961
Image predicted as [0]
Mask Val: 0.9909377098083496, Non-mask Val: 0.009062309749424458
Image predicted as [0]
Mask Val: 0.7092801332473755, Non-mask Val: 0.2907198369503021
Image predicted as [0]
Mask Val: 0.9647035598754883, Non-mask Val: 0.035296451300382614
Image predicted as [0]
Mask Val: 0.6477957367897034, Non-mask Val: 0.352204293012619
Image predicted as [0]
Mask Val: 0.9816527962684631, Non-mask Val: 0.01834724098443985
Image predicted as [0]
Mask Val: 0.9262053966522217, Non-mask Val: 0.0737946406006813
Image predicted as [0]
Mask Val: 0.8032849431037903, Non-mask Val: 0.19671504199504852
Image predicted as [0]
Mask Val: 0.17506887018680573, Non-mask Va

Mask Val: 0.5921278595924377, Non-mask Val: 0.4078720808029175
Image predicted as [0]
Mask Val: 0.979033887386322, Non-mask Val: 0.02096613496541977
Image predicted as [0]
Mask Val: 0.9010973572731018, Non-mask Val: 0.09890267997980118
Image predicted as [0]
Mask Val: 0.7223697900772095, Non-mask Val: 0.27763018012046814
Image predicted as [0]
Mask Val: 0.9717879891395569, Non-mask Val: 0.028212031349539757
Image predicted as [0]
Mask Val: 0.978574275970459, Non-mask Val: 0.021425731480121613
Image predicted as [0]
Mask Val: 0.9127081036567688, Non-mask Val: 0.0872919037938118
Image predicted as [0]
Mask Val: 0.9653298258781433, Non-mask Val: 0.034670140594244
Image predicted as [0]
Mask Val: 0.06345447152853012, Non-mask Val: 0.9365454912185669
Image predicted as [1]
Mask Val: 0.931475043296814, Non-mask Val: 0.06852494925260544
Image predicted as [0]
Mask Val: 0.8223946690559387, Non-mask Val: 0.17760540544986725
Image predicted as [0]
Mask Val: 0.8880776166915894, Non-mask Val: 0.11

Mask Val: 0.9614156484603882, Non-mask Val: 0.03858441859483719
Image predicted as [0]
Mask Val: 0.04028597101569176, Non-mask Val: 0.9597140550613403
Image predicted as [1]
Mask Val: 0.9677698612213135, Non-mask Val: 0.03223007917404175
Image predicted as [0]
Mask Val: 0.9083971381187439, Non-mask Val: 0.0916028693318367
Image predicted as [0]
Mask Val: 0.14697207510471344, Non-mask Val: 0.8530279994010925
Image predicted as [1]
Mask Val: 0.98613041639328, Non-mask Val: 0.013869608752429485
Image predicted as [0]
Mask Val: 0.971946656703949, Non-mask Val: 0.02805338054895401
Image predicted as [0]
Mask Val: 0.9594971537590027, Non-mask Val: 0.04050283506512642
Image predicted as [0]
Mask Val: 0.9750783443450928, Non-mask Val: 0.02492162398993969
Image predicted as [0]
Mask Val: 0.8606025576591492, Non-mask Val: 0.13939747214317322
Image predicted as [0]
Mask Val: 0.9931868314743042, Non-mask Val: 0.006813123822212219
Image predicted as [0]
Mask Val: 0.8696818947792053, Non-mask Val: 0

Mask Val: 0.9995336532592773, Non-mask Val: 0.00046639394713565707
Image predicted as [0]
Mask Val: 0.1995236724615097, Non-mask Val: 0.8004763722419739
Image predicted as [1]
Mask Val: 0.22010143101215363, Non-mask Val: 0.7798985838890076
Image predicted as [1]
Mask Val: 0.4359127879142761, Non-mask Val: 0.5640872716903687
Image predicted as [1]
Mask Val: 0.47242799401283264, Non-mask Val: 0.5275720953941345
Image predicted as [1]
Mask Val: 0.9952830672264099, Non-mask Val: 0.004716967698186636
Image predicted as [0]
Mask Val: 0.013864919543266296, Non-mask Val: 0.9861350655555725
Image predicted as [1]
Mask Val: 0.026937533169984818, Non-mask Val: 0.9730624556541443
Image predicted as [1]
Mask Val: 0.9785048961639404, Non-mask Val: 0.021495062857866287
Image predicted as [0]
Mask Val: 0.17763514816761017, Non-mask Val: 0.8223649263381958
Image predicted as [1]
Mask Val: 0.04050949588418007, Non-mask Val: 0.9594905376434326
Image predicted as [1]
Mask Val: 0.13370980322360992, Non-mas

Mask Val: 0.9499560594558716, Non-mask Val: 0.05004391446709633
Image predicted as [0]
Mask Val: 0.9995593428611755, Non-mask Val: 0.00044068272109143436
Image predicted as [0]
Mask Val: 0.9986159801483154, Non-mask Val: 0.0013840127503499389
Image predicted as [0]
Mask Val: 0.9979179501533508, Non-mask Val: 0.002081997459754348
Image predicted as [0]
Mask Val: 0.9892807602882385, Non-mask Val: 0.010719243437051773
Image predicted as [0]
Mask Val: 0.9531542062759399, Non-mask Val: 0.046845853328704834
Image predicted as [0]
Mask Val: 0.970811128616333, Non-mask Val: 0.02918890491127968
Image predicted as [0]
Mask Val: 0.9666762948036194, Non-mask Val: 0.033323727548122406
Image predicted as [0]
Mask Val: 0.9054266810417175, Non-mask Val: 0.09457328170537949
Image predicted as [0]
Mask Val: 0.9899264574050903, Non-mask Val: 0.010073564015328884
Image predicted as [0]
Mask Val: 0.5596643090248108, Non-mask Val: 0.4403357207775116
Image predicted as [0]
Mask Val: 0.9639275074005127, Non-m